# Imports & globals

In [20]:
import sys
sys.path.append("/home/cbarkhof/fall-2021")

In [21]:
from analysis_run_postprocessing.general_postprocessing_steps import *

%config InlineBackend.figure_format='retina'
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

DEVICE = "cuda:0"
EXPORT_DIR = f"{CODE_DIR}/analysis_run_postprocessing/run_postprocess_files"
EVALUATION_RESULT_FILE = "evaluation-results.pt"

# Fetch relevant runs

In [29]:
prefixes = ["(mdr-vae-exp 8 oct)", "(fb-vae-exp 8 oct) ", 
            "(beta-vae-exp 6 oct) ", "(inf-vae-exp 5 oct) ", 
            "(5-jan ptb-test)", "(4-jan ptb-test)", "(10-jan ptb)", 
            "(11-jan ptb)", "(19-jan ptb)", "(20-jan bmnist)", "(23-jan bmnist)"]

run_df = make_run_overview_df(prefixes)
print(len(run_df), len(run_df[run_df.image_or_language == "image"]), len(run_df[run_df.image_or_language == "language"]))
run_df.head()

146 92 54


,objective,l_rate,dataset,image_or_language,beta_beta,free_bits,mdr_value,l_mmd,decoder,run_name
INFO-VAE l_Rate 10 l_MMD 100 dec: CNN.T,INFO-VAE,10,bmnist,image,0,0,0,100,basic_deconv_decoder,"(23-jan bmnist) INFO-VAE[l_1_rate=10.0, l_2_mm..."
INFO-VAE l_Rate 10 l_MMD 10 dec: CNN.T,INFO-VAE,10,bmnist,image,0,0,0,10,basic_deconv_decoder,"(20-jan bmnist) INFO-VAE[l_1_rate=10.0, l_2_mm..."
INFO-VAE l_Rate 10 l_MMD 100 dec: PixelCNN++,INFO-VAE,10,bmnist,image,0,0,0,100,cond_pixel_cnn_pp,"(20-jan bmnist) INFO-VAE[l_1_rate=10.0, l_2_mm..."
INFO-VAE l_Rate 10 l_MMD 10 dec: PixelCNN++,INFO-VAE,10,bmnist,image,0,0,0,10,cond_pixel_cnn_pp,"(20-jan bmnist) INFO-VAE[l_1_rate=10.0, l_2_mm..."
INFO-VAE l_Rate 10 l_MMD 1 dec: PixelCNN++,INFO-VAE,10,bmnist,image,0,0,0,1,cond_pixel_cnn_pp,"(20-jan bmnist) INFO-VAE[l_1_rate=10.0, l_2_mm..."


# Check whether there are evaluation results missing

In [30]:
missing_df = overview_of_missing_analysis(run_df)
missing_df.sum()

dir               0
surprisal_data    0
encode            0
samples           0
cond_samples      0
evaluation        0
dtype: int64

# Image missing experiments

In [31]:
all_select_image = []

print("MISSING RUNS:")

print("-"*60)
print("INFO VAE")
data = run_df[(run_df.image_or_language == "image") & (run_df.objective == "INFO-VAE")]

"""
INFO VAE
    - l_rate: [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
    - l_mmd: [1, 10, 100]
"""

#print("l_mmd", data.l_mmd.unique())
#print("l_rate", data.l_rate.unique())

for l_rate in [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]:
    for l_mmd in [1, 10, 100]:
        for decoder in ["basic_deconv_decoder", "cond_pixel_cnn_pp"]:
            select = data[(data.l_mmd == l_mmd) & (data.l_rate == l_rate) & (data.decoder == decoder)]
            if len(select) == 0:
                print("l_rate", l_rate, "l_mmd", l_mmd, "decoder", decoder)
            else:
                if len(select) > 1: print("Warning multiple runs for HP setting.")
                all_select_image.append(select)
    

print("-"*60)
print("BETA VAE")

data = run_df[(run_df.image_or_language == "image") & (run_df.objective == "BETA-VAE")]

"""
BETA VAE
    - beta: [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 5.0, 10.0]
"""

#print("beta_beta", data.beta_beta.unique())

for beta in [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 5.0, 10.0]:
    for decoder in ["basic_deconv_decoder", "cond_pixel_cnn_pp"]:
        select = data[(data.beta_beta == beta) & (data.decoder == decoder)]
        if len(select) == 0:
            print("beta_beta", beta, "decoder", decoder)
        else:
            if len(select) > 1: print("Warning multiple runs for HP setting.")
            all_select_image.append(select)
            
"""
FB-VAE
    - fb: [4, 8, 16, 24, 32, 40]
"""
print("-"*60)
print("FREE BITS")
data = run_df[(run_df.image_or_language == "image") & (run_df.objective == "FB-VAE")]

#print("free_bits", data.free_bits.unique())

for free_bits in [4, 8, 16, 24, 32, 40]:
    for decoder in ["basic_deconv_decoder", "cond_pixel_cnn_pp"]:
        select = data[(data.free_bits == free_bits) & (data.decoder == decoder)]
        if len(select) == 0:
            print("free_bits", free_bits, "decoder", decoder)
        else:
            if len(select) > 1: print("Warning multiple runs for HP setting.")
            all_select_image.append(select)
            
df_all_select_image = pd.concat(all_select_image)
df_all_select_image.head()

MISSING RUNS:
------------------------------------------------------------
INFO VAE
------------------------------------------------------------
BETA VAE
------------------------------------------------------------
FREE BITS


,objective,l_rate,dataset,image_or_language,beta_beta,free_bits,mdr_value,l_mmd,decoder,run_name
INFO-VAE l_Rate 0.1 l_MMD 1 dec: CNN.T,INFO-VAE,0.1,bmnist,image,0,0,0,1,basic_deconv_decoder,"(inf-vae-exp 5 oct) INFO-VAE[l_1_rate=0.1, l_2..."
INFO-VAE l_Rate 0.1 l_MMD 1 dec: PixelCNN++,INFO-VAE,0.1,bmnist,image,0,0,0,1,cond_pixel_cnn_pp,"(inf-vae-exp 5 oct) INFO-VAE[l_1_rate=0.1, l_2..."
INFO-VAE l_Rate 0.1 l_MMD 10 dec: CNN.T,INFO-VAE,0.1,bmnist,image,0,0,0,10,basic_deconv_decoder,"(20-jan bmnist) INFO-VAE[l_1_rate=0.1, l_2_mmd..."
INFO-VAE l_Rate 0.1 l_MMD 10 dec: PixelCNN++,INFO-VAE,0.1,bmnist,image,0,0,0,10,cond_pixel_cnn_pp,"(20-jan bmnist) INFO-VAE[l_1_rate=0.1, l_2_mmd..."
INFO-VAE l_Rate 0.1 l_MMD 100 dec: CNN.T,INFO-VAE,0.1,bmnist,image,0,0,0,100,basic_deconv_decoder,"(inf-vae-exp 5 oct) INFO-VAE[l_1_rate=0.1, l_2..."


# Language missing experiments

In [32]:
all_select_language = []

print("MISSING RUNS:")

print("-"*60)
print("INFO VAE")
data = run_df[(run_df.image_or_language == "language") & (run_df.objective == "INFO-VAE")]

"""
INFO VAE
    - l_rate: [0.1, 0.5, 1.0, 2.0]
    - l_mmd: [1, 10, 100, 1000]
"""

#print("l_mmd", data.l_mmd.unique())
#print("l_rate", data.l_rate.unique())

l_rates = [0.1, 0.5, 1.0, 2.0]
l_mmds = [1, 10, 100]

for l_rate in l_rates:
    for l_mmd in l_mmds:
        decoder = 'strong_distil_roberta_decoder'
        select = data[(data.l_mmd == l_mmd) & (data.l_rate == l_rate) & (data.decoder == decoder)]
        if len(select) == 0:
            print("l_rate", l_rate, "l_mmd", l_mmd, "decoder", decoder)
        else:
            if len(select) > 1: print("Warning multiple runs for HP setting.")
            all_select_language.append(select)
    

print("-"*60)
print("BETA VAE")

data = run_df[(run_df.image_or_language == "language") & (run_df.objective == "BETA-VAE")]

"""
BETA VAE
    - beta: [0.0, 0.25, 0.5, 0.75, 1.0, 2.0]
"""

#print("beta_beta", data.beta_beta.unique())

betas = [0.0, 0.25, 0.5, 0.75, 1.0, 2.0]
for beta in betas:
    decoder = 'strong_distil_roberta_decoder'
    select = data[(data.beta_beta == beta) & (data.decoder == decoder)]
    if len(select) == 0:
        print("beta_beta", beta, "decoder", decoder)
    else:
        if len(select) > 1: print("Warning multiple runs for HP setting.")
        all_select_language.append(select)
            
"""
FB-VAE
    - fb: [4, 8, 16, 32, 64]
"""
print("-"*60)
print("FREE BITS")
data = run_df[(run_df.image_or_language == "language") & (run_df.objective == "FB-VAE")]

#print("free_bits", data.free_bits.unique())

free_bits_settings = [4, 8, 16, 32, 64]
for free_bits in free_bits_settings:
    decoder = 'strong_distil_roberta_decoder'
    select = data[(data.free_bits == free_bits) & (data.decoder == decoder)]
    if len(select) == 0:
        print("free_bits", free_bits, "decoder", decoder)
    else:
        if len(select) > 1: print("Warning multiple runs for HP setting.")
        all_select_language.append(select)

df_all_select_language = pd.concat(all_select_language)
df_all_select_language.head()

N_exp = (len(l_rates) * len(l_mmds)) + len(free_bits_settings) + len(betas)
print("N exp (l_rates x l_mmds) + free_bits_settings + betas =", N_exp)

MISSING RUNS:
------------------------------------------------------------
INFO VAE
------------------------------------------------------------
BETA VAE
------------------------------------------------------------
FREE BITS
N exp (l_rates x l_mmds) + free_bits_settings + betas = 23


In [33]:
df_all_select = pd.concat([df_all_select_image, df_all_select_language])
print(len(df_all_select_image), len(df_all_select_language))
df_all_select.to_csv("final_selection_runs.csv")
df_all_select

64 23


,objective,l_rate,dataset,image_or_language,beta_beta,free_bits,mdr_value,l_mmd,decoder,run_name
INFO-VAE l_Rate 0.1 l_MMD 1 dec: CNN.T,INFO-VAE,0.1,bmnist,image,0,0,0,1,basic_deconv_decoder,"(inf-vae-exp 5 oct) INFO-VAE[l_1_rate=0.1, l_2..."
INFO-VAE l_Rate 0.1 l_MMD 1 dec: PixelCNN++,INFO-VAE,0.1,bmnist,image,0,0,0,1,cond_pixel_cnn_pp,"(inf-vae-exp 5 oct) INFO-VAE[l_1_rate=0.1, l_2..."
INFO-VAE l_Rate 0.1 l_MMD 10 dec: CNN.T,INFO-VAE,0.1,bmnist,image,0,0,0,10,basic_deconv_decoder,"(20-jan bmnist) INFO-VAE[l_1_rate=0.1, l_2_mmd..."
INFO-VAE l_Rate 0.1 l_MMD 10 dec: PixelCNN++,INFO-VAE,0.1,bmnist,image,0,0,0,10,cond_pixel_cnn_pp,"(20-jan bmnist) INFO-VAE[l_1_rate=0.1, l_2_mmd..."
INFO-VAE l_Rate 0.1 l_MMD 100 dec: CNN.T,INFO-VAE,0.1,bmnist,image,0,0,0,100,basic_deconv_decoder,"(inf-vae-exp 5 oct) INFO-VAE[l_1_rate=0.1, l_2..."
...,...,...,...,...,...,...,...,...,...,...
FB-VAE 4 dec: Strong roBERTa,FB-VAE,0,ptb,language,0,4,0,0,strong_distil_roberta_decoder,(19-jan ptb) FB-VAE[4.0-pd=False] | STRONG | N...
FB-VAE 8 dec: Strong roBERTa,FB-VAE,0,ptb,language,0,8,0,0,strong_distil_roberta_decoder,(19-jan ptb) FB-VAE[8.0-pd=False] | STRONG | N...
FB-VAE 16 dec: Strong roBERTa,FB-VAE,0,ptb,language,0,16,0,0,strong_distil_roberta_decoder,(10-jan ptb) FB-VAE[16.0-pd=False] | STRONG | ...
FB-VAE 32 dec: Strong roBERTa,FB-VAE,0,ptb,language,0,32,0,0,strong_distil_roberta_decoder,(10-jan ptb) FB-VAE[32.0-pd=False] | STRONG | ...
